##**ROPTA**



# Nougat OCR

In [ ]:
# Instalamos la librería Nougat para procesar los pdf
from IPython import display
import os
!pip install nougat-ocr

In [ ]:
# Llamamos a un pdf que conseguimos en arxiv y le pedimos que lo descargue
!curl -o quantum_physics.pdf https://arxiv.org/pdf/2302.00078.pdf

In [ ]:
# Acá le pedimos que lo procese y lo guarde en una carpeta
!nougat pdf '/content/Nougat/Input/Estructura Fina.pdf' --out '/content/Nougat/Output'

In [ ]:
# Aquí mostramos el resultado en formato mmd
display.Markdown('/content/Nougat/output/*.mmd')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Aquí mostramos el resultado en formato Latex
display.Latex('ToOverleaf.mmd')

In [ ]:
# Repetimos el proceso anterior para otro pdf que está formado por fotos de un libro
!curl -o dirac.pdf https://www.informationphilosopher.com/solutions/scientists/dirac/Fund_QM_1925.pdf

In [ ]:
!nougat pdf '/content/Nougat/Input/ScannedPdf.pdf' --out 'output'

In [ ]:
display.Markdown('/content/Nougat/Output/ToOverleaf.mmd')

Es posible hacer este proceso para varios archivos a la vez, sin embargo, requiere de mucha capacidad de computo por lo que omitimos esta función.

In [ ]:
# Aquí subimos un archivo local y lo procesamos
!nougat pdf '/content/CamScanner 10-21-2023 21.41.pdf' --out 'output' --no-skipping

In [ ]:
display.Markdown('/content/CamScanner 10-21-2023 21.41.pdf.mmd')

# RESUMEN DE ROPTA

Instalación

In [ ]:
!pip install langchain
!pip install openai
!pip install chromadb
!pip install pypdf
!pip install tiktoken

In [4]:
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader
from langchain import OpenAI, PromptTemplate
import glob

APi Key OpenAI

In [9]:
#Aqui Usuario añade el Api Key de OpenAI

llm = OpenAI(temperature=0.2 , openai_api_key="INSERTE_SU_API_KEY_AQUI")

Función Resumen

In [6]:
#Definición Resumen Personalizado
def custom_summary(pdf_folder, custom_prompt):
    summaries = []
    for pdf_file in glob.glob(pdf_folder + "/*.pdf"):
        loader = PyPDFLoader(pdf_file)
        docs = loader.load_and_split()
        prompt_template = custom_prompt + """

        {text}

        SUMMARY:"""
        PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
        chain = load_summarize_chain(llm, chain_type="map_reduce",
                                    map_prompt=PROMPT, combine_prompt=PROMPT)
        summary_output = chain({"input_documents": docs},return_only_outputs=True)["output_text"]
        summaries.append(summary_output)

    return summaries

Ejecución Resumen

In [ ]:
CUSTOM_PROMPT = "Write a detailed summary of the document. The summary should be approximately 25% of the length of the original text"
custom_summaries = custom_summary("./pdfs", custom_prompt=CUSTOM_PROMPT)

#Guardar todos los resúmenes a un solo .txt
with open("custom_summaries.txt", "w") as f:
  for summary in custom_summaries:
    f.write(summary + "\n"*3)

# PREGUNTAS AL ROPTA

In [ ]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.document_loaders import PyPDFDirectoryLoader
import os

# Establece la clave de la API como una variable de entorno
os.environ['OPENAI_API_KEY'] = 'INSERTE_SU_API_KEY_AQUI'
# Python!
loader = PyPDFDirectoryLoader("/content/pdfs/")

docs = loader.load()

# Create the vector store index
index = VectorstoreIndexCreator().from_loaders([loader])

# Realiza una consulta
query = "¿Cuál es la relación entre las transformaciones lineales y la teoría de grupos?"
response = index.query(query)

print(f"Respuesta: {response}")